In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

In [2]:
inp=pd.read_csv("../PJ sensor.csv",usecols=[6,7,10,11])
out=pd.read_csv("../PJ sensor.csv",usecols=[2,3,4,5,8,9])

In [3]:
inp.head(5)

,Temperature,Relative Humidity,date,Time
0,31,65,191213,175844
1,31,65,191213,175904
2,31,65,191213,175924
3,31,65,191213,175944
4,31,66,191213,180004


In [4]:
out.head(5)

,NO2,O3,CO,SO2,PM2.5,PM10
0,-15.075377,35.714286,1766.233766,148.484848,11.229150,17.323938
1,-20.100503,41.208791,1733.766234,142.424242,9.413914,24.967407
2,-15.075377,43.956044,1711.038961,136.363636,10.350110,56.781536
3,0.000000,57.692308,1688.311688,139.393939,10.488765,30.371115
4,-20.100503,54.945055,1681.818182,130.303030,12.353048,188.026886


In [5]:
inp.tail(10)

,Temperature,Relative Humidity,date,Time
1627523,34,60,211111,160531
1627524,34,60,211111,160551
1627525,34,60,211111,160611
1627526,34,60,211111,160631
1627527,34,60,211111,160651
1627528,34,60,211111,160711
1627529,34,60,211111,160731
1627530,34,60,211111,160751
1627531,34,60,211111,160811
1627532,34,60,211111,160831


In [6]:
inp.shape
out.shape

(1627533, 6)

In [7]:
inp=np.array(inp)
out=np.array(out)

In [8]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(inp)
Y1=scaler_obj.fit_transform(out)

warnings.filterwarnings(action='ignore', category=UserWarning)

In [7]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.2,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

#creating object of sgboostregressor
model1=MultiOutputRegressor(LinearRegression())

model_fit1=model1.fit(x_train, y_train)
%time model_fit1
print("Model training is Done!!")

filename1 = 'linearregression.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

Wall time: 0 ns
Model training is Done!!


In [8]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')


r2 score on train data 0.2948116798857159
r2 score on test data 0.2929796878382063
Mean Absolute Error: 0.01391719554120551
Mean Squared Error: 0.0005747420137439317
Root Mean Squared Error: 0.023973777627731757
 



In [9]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("linearregression.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[0.45424568 0.57100102 0.04166631 0.00847074 0.05707396 0.00425943]
 [0.41350837 0.55205009 0.03457935 0.00853196 0.04844683 0.00387805]
 [0.46630211 0.57537136 0.04236458 0.00834588 0.05742094 0.0044362 ]
 ...
 [0.40383026 0.55251122 0.03414557 0.00872219 0.04674389 0.00365964]
 [0.39652864 0.54442455 0.03096275 0.00863495 0.04512389 0.00395007]
 [0.4430619  0.5484787  0.0391053  0.00851174 0.06721492 0.00531713]]


R2 Score :  0.2932843874660527


In [ ]:
#completed